In [1]:
!pip install tensorflow
import tensorflow as tf
print(tf.__version__)

2.11.0


In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
IMAGE_SIZE = [224, 224]

train_path = 'Dataset/train'
valid_path = 'Dataset/test'

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
for layer in vgg.layers:
  layer.trainable = False

In [6]:
folders = glob('Dataset_Apple/train/*')


In [7]:
folders

['Dataset_Apple/train\\fresh Apples', 'Dataset_Apple/train\\rotten Apples']

In [8]:
x = Flatten()(vgg.output)

In [9]:
len(folders)

2

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

In [11]:
model = Model(inputs=vgg.input, outputs=prediction)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
training_set = train_datagen.flow_from_directory('Dataset_Apple/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4035 images belonging to 2 classes.


In [18]:
test_set = test_datagen.flow_from_directory('Dataset_Apple/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 996 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_41872\3484128432.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/2
127/127 [==============================] - 5149s 41s/step - loss: 0.2535 - accuracy: 0.8954 - val_loss: 0.0797 - val_accuracy: 0.9759
Epoch 2/2
127/127 [==============================] - ETA: 0s - loss: 0.1054 - accuracy: 0.9608  

In [ ]:

scratch_preds = model.predict(test_set)
scratch_pred_classes = np.argmax(scratch_preds, axis=1)

# scratch_acc = accuracy_score(true_classes, scratch_pred_classes)
# print("From Scratch Model Accuracy with Fine-Tuning: {:.2f}%".format(scratch_acc * 100))


In [ ]:
acc = r.history['accuracy']
val_acc = r.history['val_accuracy']
loss = r.history['loss']
val_loss = r.history['val_loss']
epochs_range = range(len(acc))

In [ ]:
plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc,"go-", label='Training Accuracy')
plt.plot(epochs_range, val_acc,"ro-", label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('AccurateValue_acc_points')

In [ ]:
plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss,"go-", label='Training Loss')
plt.plot(epochs_range, val_loss,"ro-", label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()
plt.savefig('LossValue_loss_points')

In [ ]:
pip install opencv-python

In [ ]:
import cv2
image = cv2.imread('Dataset_Apple/single_prediction/rotten3.jpeg')
image_resized = cv2.resize(image, (224,224))
image = np.expand_dims(image_resized,axis=0)
print(image.shape)

In [ ]:
predict = model.predict(image)

In [ ]:
print(predict)

In [ ]:
if predict[0][0] > predict[0][1]:
    print("fresh")
else:
    print("rotten")

In [ ]:
y_pred =  model.predict_generator(test_set)

In [ ]:
threshold = 0.5
y_final = np.where(y_pred > threshold, 1,0)

In [ ]:
y_final.size

In [ ]:
Y_val = folders[0][0]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

test_steps_per_epoch = np.math.ceil(test_set.samples / test_set.batch_size)

predictions = model.predict_generator(test_set, steps=test_steps_per_epoch)

# Get most likely class
predicted_classes = [1 * (x[0]>=0.5) for x in predictions]

# 2.Get ground-truth classes and class-labels
true_classes = test_set.classes
class_labels = list(test_set.class_indices.keys()) 

# 3. Use scikit-learn to get statistics
import sklearn
from sklearn.metrics import confusion_matrix,classification_report

print(class_labels)
cm = confusion_matrix(test_set.classes, predicted_classes) 
print(cm)

report = sklearn.metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 

In [ ]:
class_names = test_set.class_indices.keys()

In [ ]:
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()
# tick_marks = np.arange(len(classes))
# plt.xticks(test_set, classes, rotation=45)
# plt.yticks(predicted_classes, classes)
plt.xlabel('Predicted label')
plt.ylabel('True label')

# add labels to each cell
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, cm[i, j],
             ha="center", va="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()
